In [20]:
%matplotlib inline
import os, sys

import matplotlib
matplotlib.style.use("ggplot")
import matplotlib.pyplot as plt
import pandas as pd
import yaml

import pcse
from pcse.models import Wofost72_WLP_FD, Wofost80_NWLP_FD_beta, LINTUL3
from pcse.fileinput import CABOFileReader, YAMLCropDataProvider, PCSEFileReader
from pcse.db import NASAPowerWeatherDataProvider
from pcse.util import WOFOST72SiteDataProvider, WOFOST80SiteDataProvider
from pcse.base import ParameterProvider
from pcse.fileinput import YAMLAgroManagementReader

import numpy as np

In [21]:
data_dir = os.path.join(os.getcwd(), 'farm_gym/envs/env_data/')

In [22]:
crop_params_dir = os.path.join(data_dir, 'crop_params/')
crop = YAMLCropDataProvider(crop_params_dir)

YAMLCropDataProvider - Failed to load cache file: [Errno 2] No such file or directory: 'barley'


In [23]:
soil_data = os.path.join(data_dir, 'SOILD', 'EC4.NEW')
soil = CABOFileReader(soil_data)

In [24]:
sited = WOFOST72SiteDataProvider(WAV=10)

In [25]:
parameterprovider = ParameterProvider(soildata=soil, cropdata=crop, sitedata=sited)

In [26]:
def get_weatherdataprovider(lat, long):
    location = (lat, long)
    return pcse.db.NASAPowerWeatherDataProvider(*location)
wdp = get_weatherdataprovider(-33.385300, 148.007904)
#wdp = get_weatherdataprovider(51.970, 5.670)
print(wdp)

Weather data provided by: NASAPowerWeatherDataProvider
--------Description---------
NASA/POWER CERES/MERRA2 Native Resolution Daily Data
----Site characteristics----
Elevation:  291.1
Latitude:  -33.385
Longitude: 148.008
Data available for 1984-01-01 - 2022-07-21
Number of missing days: 1



In [27]:
agro_dir = os.path.join(os.getcwd(), "farm_gym/envs/env_data/agro/")
agromanagement_file = os.path.join(agro_dir, 'wofost_wheat.yaml')
agromanagement = YAMLAgroManagementReader(agromanagement_file)
print(agromanagement)

!!python/object/new:pcse.fileinput.yaml_agro_loader.YAMLAgroManagementReader
listitems:
- 2006-01-01:
    CropCalendar: null
    StateEvents: null
    TimedEvents: null



In [28]:
wofost = Wofost72_WLP_FD(parameterprovider, wdp, agromanagement)

In [29]:
wofost.run(days=10)

In [30]:
output = pd.DataFrame(wofost.get_output()).set_index("day")
output = output.fillna(value=np.nan)


In [31]:
print(output.index[-1])

2006-01-01


In [32]:
print(output)

            DVS  LAI  TAGP  TWSO  TWLV  TWST  TWRT  TRA  RD   SM  WWLOW
day                                                                    
2006-01-01  NaN  NaN   NaN   NaN   NaN   NaN   NaN  NaN NaN  0.4   46.0
